In [1]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Agent Builder Search Evaluation

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/vertex_search_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fvertex_search_evaluation.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/vertex_search_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/vertex_search_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | | | |
|-|-|-|-|
|Author(s) | [Rupjit Chakraborty](https://github.com/lazyprgmr)

# Overview

This notebook demonstrates evaluation of search results for Vertex AI Agent Builder Search

## Getting Started

### Install required packages


In [ ]:
%pip install opendatasets==0.1.22 vertexai==1.70.0 pandas==2.2.3 google-cloud-discoveryengine==0.13.2 ndjson==0.3.1

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After its restarted, continue to the next step.


In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the cell below to authenticate your environment.


In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()
    print("Authenticated")

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).


In [ ]:
PROJECT_ID = ""  # @param {type:"string"}
LOCATION = ""  # @param {type:"string"}
MODEL_NAME = ""  # @param {type:"string"}
BUCKET_URI = ""  # @param {type:"string"}

## Import packages

In [ ]:
import os

from google.api_core.client_options import ClientOptions
import google.auth
import google.auth.transport.requests
from google.cloud import discoveryengine, discoveryengine_v1
import ndjson
import opendatasets as od
import pandas as pd
import requests
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Downloading data from Kaggle

In order to use the Kaggle public API, you must first authenticate using an API token. Go to the `Account` tab of your user profile and select `Create New Token`. This will trigger the download of `kaggle.json`, a file containing your API credentials.

[Source](https://www.kaggle.com/docs/api)

Once the `kaggle.json` has been downloaded, move it to this directory where this notebook is present.

In [ ]:
# View the username and key

! cat "kaggle.json"

In [ ]:
# Download the data from Kaggle by providing the username and password from "kaggle.json"

dataset = "https://www.kaggle.com/datasets/muhammetvarl/seo-sample-data"
od.download(dataset)

In [ ]:
# The downloaded data should be visible in a folder (seo-sample-data)

os.listdir()

In [ ]:
# Load the csv file

data = pd.read_csv("seo-sample-data/SEO_data.csv")

In [ ]:
# Checkout the data

data.head()

In [ ]:
# Identify the number of unique search queries in the data and the number of search results for each

data.groupby(["words"]).count()

In [ ]:
# For this demonstration we will just take 'five' search terms and 'ten' associated result links
num_terms = 5
# select first five unique search terms
search_terms = data["words"].unique()[:num_terms].tolist()
# filter dataframe based on first five unique search terms
selected_data = data[data["words"].isin(search_terms)]
# select results upto rank 10 for each search term
selected_data = selected_data[selected_data["rank"].isin(range(1, 11))]
# keep only the words and links column
selected_data = selected_data.loc[:, ["words", "links"]]
# view the final data
selected_data.head()

In [ ]:
len(selected_data)

## Creating a datastore

**NOTE**: Make sure the Vertex AI Agent Builder API is [enabled](https://cloud.google.com/generative-ai-app-builder/docs/before-you-begin#turn-on-discovery-engine) before executing the next cell

In [ ]:
DATA_STORE_ID = ""
DS_LOC = ""

In [ ]:
# create a data store
# Ref: https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#website
# Instructions for enabling the search api


def create_data_store() -> str:
    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    client_options = (
        ClientOptions(api_endpoint=f"{DS_LOC}-discoveryengine.googleapis.com")
        if DS_LOC != "global"
        else None
    )

    # Create a client
    client = discoveryengine.DataStoreServiceClient(client_options=client_options)

    # The full resource name of the collection
    # e.g. projects/{project}/locations/{location}/collections/default_collection
    parent = client.collection_path(
        project=PROJECT_ID,
        location=DS_LOC,
        collection="default_collection",
    )

    data_store = discoveryengine.DataStore(
        display_name="Search Evaluation Demo",
        # Options: GENERIC, MEDIA
        industry_vertical=discoveryengine.IndustryVertical.GENERIC,
        # Options: SOLUTION_TYPE_RECOMMENDATION, SOLUTION_TYPE_SEARCH, SOLUTION_TYPE_CHAT, SOLUTION_TYPE_GENERATIVE_CHAT
        solution_types=[discoveryengine.SolutionType.SOLUTION_TYPE_SEARCH],
        # TODO(developer): Update content_config based on data store type.
        # Options: NO_CONTENT, CONTENT_REQUIRED, PUBLIC_WEBSITE
        content_config=discoveryengine.DataStore.ContentConfig.PUBLIC_WEBSITE,
    )

    request = discoveryengine.CreateDataStoreRequest(
        parent=parent,
        data_store_id=DATA_STORE_ID,
        data_store=data_store,
        # Optional: For Advanced Site Search Only
        # create_advanced_site_search=True,
    )

    # Make the request
    operation = client.create_data_store(request=request)

    print(f"Waiting for operation to complete: {operation.operation.name}")
    response = operation.result()

    # After the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine.CreateDataStoreMetadata(operation.metadata)

    # Handle the response
    print(response)
    print(metadata)

    return operation.operation.name

In [ ]:
create_data_store()

## Importing data into datastore

In [ ]:
# Import data into the datastore
# Ref: https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#website

# NOTE: Do not include http or https protocol in the URI pattern
# eg. uri_pattern = "cloud.google.com/generative-ai-app-builder/docs/*"
uri_patterns = selected_data["links"].values.tolist()
print(f"Number of urls to index: {len(uri_pattern)}")

#  For more information, refer to:
# https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
client_options = (
    ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
    if DS_LOC != "global"
    else None
)

# Create a client
client = discoveryengine_v1.SiteSearchEngineServiceClient(client_options=client_options)

# The full resource name of the data store
# e.g. projects/{project}/locations/{location}/dataStores/{data_store_id}
site_search_engine = client.site_search_engine_path(
    project=PROJECT_ID, location=DS_LOC, data_store=DATA_STORE_ID
)


target_sites = []
for uri_pattern in uri_patterns:
    # Remove the protocol information
    if uri_pattern.startswith("https://"):
        uri_pattern = uri_pattern.replace("https://", "")
    elif uri_pattern.startswith("http://"):
        uri_pattern = uri_pattern.replace("http://", "")

    # Create the TargetSite object
    target_site = discoveryengine_v1.TargetSite(
        provided_uri_pattern=uri_pattern,
        # Options: INCLUDE, EXCLUDE
        type_=discoveryengine_v1.TargetSite.Type.INCLUDE,
        exact_match=False,
    )

    # Create a CreateTargetSiteRequest object from the TargetSite object and then save it in a list
    target_sites.append(
        discoveryengine_v1.CreateTargetSiteRequest(
            parent=f"projects/{PROJECT_ID}/locations/{DS_LOC}/collections/default_collection/dataStores/{DATA_STORE_ID}/siteSearchEngine",
            target_site=target_site,
        )
    )

# Create the batch request object (0 < batch size <= 20)
batch_size = 10
for idx in range(0, len(target_sites), batch_size):
    target_sites_batch = target_sites[idx : idx + batch_size]
    batch_create_site_req = discoveryengine_v1.BatchCreateTargetSitesRequest(
        parent=f"projects/{PROJECT_ID}/locations/{DS_LOC}/collections/default_collection/dataStores/{DATA_STORE_ID}/siteSearchEngine",
        requests=target_sites_batch,
    )

    # Make the request
    operation = client.batch_create_target_sites(request=batch_create_site_req)

    print(f"Waiting for operation to complete: {operation.operation.name}")
    response = operation.result()

    # After the operation is complete,
    # get information from operation metadata
    metadata = discoveryengine_v1.CreateTargetSiteMetadata(operation.metadata)

    # Handle the response
    print(response)
    print(metadata)

## Preparing data for evaluation

In [ ]:
eval_dict = {}
search_terms = selected_data["words"].unique()
for s in search_terms:
    eval_dict[s] = selected_data[selected_data["words"] == s]["links"].tolist()

In [ ]:
eval_dict

In [ ]:
def create_query_datum(question: str, uris: list[str]) -> dict:
    """Create a data point for search evaluation.

    Args:
        question: The test question
        uris: The links containing the answer for the test question

    Returns:
        A dict in a format defined in
        https://cloud.google.com/generative-ai-app-builder/docs/evaluate-search-quality#bq-gcs-templates
    """
    query_set = {"queryEntry": {"query": "", "targets": []}}
    query_set["queryEntry"]["query"] = question
    query_set["queryEntry"]["targets"] = [{"uri": uri} for uri in uris]
    return query_set

In [ ]:
# Convert the test data to n-dimensional json


test_data_formatted = []

for ques, refs in eval_dict.items():
    test_data_formatted.append(create_query_datum(ques, refs))

with open("test_data.ndjson", "w") as f_handle:
    ndjson.dump(test_data_formatted, f_handle)

In [ ]:
# Move the n-dimensional json to GCS

! gsutil cp test_data.ndjson $BUCKET_URI

## Query set creation

In [ ]:
def get_access_token():
    creds, project = google.auth.default()

    # creds.valid is False, and creds.token is None
    # Need to refresh credentials to populate those
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)

    return creds.token

In [ ]:
QUERY_SET_ID = ""
DISPLAY_NAME = ""

headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}
query_set_uri = f"https://discoveryengine.googleapis.com/v1beta/projects/{PROJECT_ID}/locations/{DS_LOC}/sampleQuerySets?sampleQuerySetId={QUERY_SET_ID}"
data = {"displayName": DISPLAY_NAME}
query_set_creation_response = requests.post(
    url=query_set_uri,
    headers=headers,
    json=data,
)

# Check the response status code
if query_set_creation_response.status_code == 200:
    print(query_set_creation_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {query_set_creation_response.status_code}")
    print(query_set_creation_response.text)  # Print the error message

## Importing the sample query data

In [ ]:
headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}
query_set_import_uri = f"https://discoveryengine.googleapis.com/v1beta/projects/{PROJECT_ID}/locations/{DS_LOC}/sampleQuerySets/{QUERY_SET_ID}/sampleQueries:import"
data = {
    "gcsSource": {
        "inputUris": [f"{BUCKET_URI}/test_data.ndjson"],
    },
    "errorConfig": {"gcsPrefix": f"{BUCKET_URI}/errors/"},
}
sample_data_import_response = requests.post(
    url=query_set_import_uri,
    headers=headers,
    json=data,
)

# Check the response status code
if sample_data_import_response.status_code == 200:
    print(sample_data_import_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {sample_data_import_response.status_code}")
    print(sample_data_import_response.text)  # Print the error message

In [ ]:
sample_data_import_response.json()["name"]

### Check status of data import operation

In [ ]:
headers = {
    "Authorization": f"Bearer {get_access_token()}",
}
data_import_status_uri = f"https://discoveryengine.googleapis.com/v1beta/{sample_data_import_response.json()['name']}"
import_status_response = requests.get(
    url=data_import_status_uri,
    headers=headers,
)

# Check the response status code
if import_status_response.status_code == 200:
    print(import_status_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {import_status_response.status_code}")
    print(import_status_response.text)  # Print the error message

## Search quality evaluation

Before running the next cell [create a search app and connect it to the datastore](https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es).
Then copy the `ID` of the app and assign it to the `APP_ID` constant below.

In [ ]:
APP_ID = ""

headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}
search_evaluation_uri = f"https://discoveryengine.googleapis.com/v1beta/projects/{PROJECT_ID}/locations/{DS_LOC}/evaluations"
data = {
    "evaluationSpec": {
        "querySetSpec": {
            "sampleQuerySet": f"projects/{PROJECT_ID}/locations/{DS_LOC}/sampleQuerySets/{QUERY_SET_ID}"
        },
        "searchRequest": {
            "servingConfig": f"projects/{PROJECT_ID}/locations/{DS_LOC}/collections/default_collection/engines/{APP_ID}/servingConfigs/default_search"
        },
    }
}
search_eval_response = requests.post(
    url=search_evaluation_uri,
    headers=headers,
    json=data,
)

# Check the response status code
if search_eval_response.status_code == 200:
    print(search_eval_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {search_eval_response.status_code}")
    print(search_eval_response.text)  # Print the error message

In [ ]:
search_eval_response.json()["response"]["name"]

In [ ]:
headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}

evaluation_status_url = f"https://discoveryengine.googleapis.com/v1beta/{search_eval_response.json()['response']['name']}"
eval_status_response = requests.get(url=evaluation_status_url, headers=headers)

# Check the response status code
if eval_status_response.status_code == 200:
    print(eval_status_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {eval_status_response.status_code}")
    print(eval_status_response.text)  # Print the error message

## Results

### Aggregate results

In [ ]:
headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}

search_results_url = f"https://discoveryengine.googleapis.com/v1beta/{search_eval_response.json()['response']['name']}"
aggregate_result_response = requests.get(url=search_results_url, headers=headers)

# Check the response status code
if aggregate_result_response.status_code == 200:
    print(aggregate_result_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {aggregate_result_response.status_code}")
    print(aggregate_result_response.text)  # Print the error message

In [ ]:
aggregate_result_response.json()["qualityMetrics"]

### Query level results

In [ ]:
headers = {
    "Authorization": f"Bearer {get_access_token()}",
    "Content-Type": "application/json",
    "X-Goog-User-Project": PROJECT_ID,
}

eval_id = search_eval_response.json()["response"]["name"].split("/")[-1]
search_results_detailed_url = f"https://discoveryengine.googleapis.com/v1beta/projects/{PROJECT_ID}/locations/global/evaluations/{eval_id}:listResults"
detailed_result_response = requests.get(
    url=search_results_detailed_url, headers=headers
)

# Check the response status code
if detailed_result_response.status_code == 200:
    print(detailed_result_response.json())  # Print the JSON response
else:
    print(f"Request failed with status code: {detailed_result_response.status_code}")
    print(detailed_result_response.text)  # Print the error message